![Ayahuasca vs Drugs](../Visualizations/Aya.png)

## Problem Statement
Collect posts from 2 subreddits from www.reddit.com, DrugNerd and Ayahuasca, Using Reddit's API. Then using NLP, to train a classifier on which subreddit a given post came from. The accuracy might tell us that people consider Ayahuasca as a sacred medicine or just a drug. We will see if they talking about Ayahuasca differently then about any other drugs.

In [1]:
import datetime
import pandas as pd
import numpy as np
import requests
import time
import regex as re
import nltk
from sklearn.utils import shuffle
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup
np.random.seed(42)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Collecting data
 
Here we are going to collect posts from 2 subreddits from www.reddit.com, DrugNerds and Ayahuasca, Using Reddit's API.

In [2]:
url = 'https://www.reddit.com/r/Ayahuasca.json'
res = requests.get(url, headers = {'User-agent': "Infinity Browser"})

In [3]:
res.status_code

200

In [4]:
soup = BeautifulSoup(res.content, 'lxml')

In [5]:
posts = res.json()
posts.keys

<function dict.keys>

In [6]:
posts['data'].keys()

dict_keys(['modhash', 'dist', 'children', 'after', 'before'])

In [7]:
posts['data']['dist']

27

In [8]:
post_data = posts['data']

In [9]:
post_data['after']

't3_aqy1o7'

In [10]:
len(post_data['children'])

27

In [11]:
len(post_data['children'][0]['data'].keys())

96

In [12]:
list_posts = []

for post in post_data['children']:
    list_posts.append(post['data'])

In [13]:
df_posts = pd.DataFrame(list_posts)

In [14]:
df_posts.head()

,approved_at_utc,approved_by,archived,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,...,thumbnail_height,thumbnail_width,title,ups,url,user_reports,view_count,visited,whitelist_status,wls
0,None,None,True,clueso87,None,None,[],None,None,None,...,NaN,NaN,Ayahuasca FAQ,41,https://www.reddit.com/r/Ayahuasca/comments/7b...,[],None,False,None,None
1,None,None,True,clueso87,None,None,[],None,None,None,...,NaN,NaN,"Reminder: do not advertise retreat centers, bu...",34,https://www.reddit.com/r/Ayahuasca/comments/7k...,[],None,False,None,None
2,None,None,False,wingedmongoose,None,None,[],None,None,None,...,105.0,140.0,Ayahuasca at Home,9,https://www.peacefulmountainway.org/single-pos...,[],None,False,None,None
3,None,None,False,SumKallMeTIM,None,None,[],None,None,None,...,NaN,NaN,Travel to Iquitos and Lima Tips?,2,https://www.reddit.com/r/Ayahuasca/comments/as...,[],None,False,None,None
4,None,None,False,-seoul-,None,None,[],None,None,None,...,NaN,NaN,Making caapi tea,3,https://www.reddit.com/r/Ayahuasca/comments/ar...,[],None,False,None,None


In [15]:
df_posts.shape

(27, 96)

In [16]:
after_json = requests.get('https://www.reddit.com/r/ayahuasca.json', 
                       headers = {'User-agent': "Infinity Browser"},
                        params={'after' : None, 'limit':'1000'}).json()

In [17]:
posts = [post['data'] for post in after_json['data']['children']]

In [18]:
after_key = after_json['data']['after']

In [19]:
after_key = None

posts_list = []

for i in range(10):
    after_json = requests.get('https://www.reddit.com/r/ayahuasca.json', 
                       headers = {'User-agent': "Infinity Browser"},
                        params={'after' : after_key, 'limit':'1000'}).json()
    after_key = after_json['data']['after']
    
    posts_list.extend([post['data'] for post in after_json['data']['children']])
    
df = pd.DataFrame(posts_list)

df.shape
    

(1002, 100)

In [20]:
df_aya = df
df_aya.head()

,approved_at_utc,approved_by,archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,...,thumbnail_height,thumbnail_width,title,ups,url,user_reports,view_count,visited,whitelist_status,wls
0,None,None,True,clueso87,NaN,None,None,[],None,None,...,NaN,NaN,Ayahuasca FAQ,44,https://www.reddit.com/r/Ayahuasca/comments/7b...,[],None,False,None,None
1,None,None,True,clueso87,NaN,None,None,[],None,None,...,NaN,NaN,"Reminder: do not advertise retreat centers, bu...",37,https://www.reddit.com/r/Ayahuasca/comments/7k...,[],None,False,None,None
2,None,None,False,wingedmongoose,NaN,None,None,[],None,None,...,105.0,140.0,Ayahuasca at Home,9,https://www.peacefulmountainway.org/single-pos...,[],None,False,None,None
3,None,None,False,SumKallMeTIM,NaN,None,None,[],None,None,...,NaN,NaN,Travel to Iquitos and Lima Tips?,2,https://www.reddit.com/r/Ayahuasca/comments/as...,[],None,False,None,None
4,None,None,False,-seoul-,NaN,None,None,[],None,None,...,NaN,NaN,Making caapi tea,3,https://www.reddit.com/r/Ayahuasca/comments/ar...,[],None,False,None,None


In [21]:
after_key = None

posts_list = []

for i in range(10):
    after_json = requests.get('https://www.reddit.com/r/DrugNerds.json', 
                       headers = {'User-agent': "Infinity Browser"},
                        params={'after' : after_key, 'limit':'1000'}).json()
    after_key = after_json['data']['after']
    
    posts_list.extend([post['data'] for post in after_json['data']['children']])
    
df_drug = pd.DataFrame(posts_list)

df_drug.shape

(946, 99)

### Once we collected about 1000 posts from each subreddit - we want to put them in dataframe, using only columns we are interested in.

In [22]:
df_aya = df_aya[['selftext' ,'subreddit', 'title']]
df_aya.head()

,selftext,subreddit,title
0,*This is intended to be a FAQ for people who w...,Ayahuasca,Ayahuasca FAQ
1,We have a no-advertisement-rule on this subred...,Ayahuasca,"Reminder: do not advertise retreat centers, bu..."
2,,Ayahuasca,Ayahuasca at Home
3,"Hello, \nGoing to a reputable place outside Iq...",Ayahuasca,Travel to Iquitos and Lima Tips?
4,I have some 15x harmala infused caapi leaves. ...,Ayahuasca,Making caapi tea


In [23]:
df_drug = df_drug[['selftext' ,'subreddit', 'title']]
df_drug.head()

,selftext,subreddit,title
0,,DrugNerds,Announcement: /r/AskDrugNerds: a place to ask ...
1,You need to comment your discord username here...,DrugNerds,Comment here to become eligible for the Drugne...
2,DNP is a metabolic poision which disrupts ener...,DrugNerds,UK Gov Advisory Council on Drugs (ACMD) issues...
3,[https://www.ncbi.nlm.nih.gov/pubmed/11267629...,DrugNerds,"""Chronic inositol increases striatal D(2) rece..."
4,https://en.m.wikipedia.org/wiki/Benzydamine\n\...,DrugNerds,Benzydamine: a hallucinogen with a completely ...


# EDA of the words in each DF

I compared the most commonly used words in each dataset, so we can see the difference between how people talk about Ayahuasca and drugs.  

![](../Visualizations/Aya_drugs.png)
**Note:** There is no more code in this notebook for this two sets, because I used some specific stop words for subsequent modeling.

### Now we are going to merge both sets together and shuffle it and save into .csv file.

In [328]:
df = df_aya.append(df_drug)
df.shape

(1955, 3)

In [329]:
df = shuffle(df)

In [330]:
df = df.reset_index()

In [331]:
df.drop(columns=['index'], inplace=True)
df.head()

,selftext,subreddit,title
0,,DrugNerds,The effects of vitamin B12 on the brain damage...
1,"After you took Aya, did you want to quit? Did ...",Ayahuasca,How Did You Feel About Your Work After Aya?
2,"Good Morning!\n\nHave done a lot of research, ...",Ayahuasca,Colorado
3,,DrugNerds,Vitamin D4 in Mushrooms
4,"Hi, sorry to bring this up, I am planning to t...",Ayahuasca,Is Pulse tours still open for business?


In [332]:
df.to_csv('aya_drug_data.csv')

In [333]:
df.head()

,selftext,subreddit,title
0,,DrugNerds,The effects of vitamin B12 on the brain damage...
1,"After you took Aya, did you want to quit? Did ...",Ayahuasca,How Did You Feel About Your Work After Aya?
2,"Good Morning!\n\nHave done a lot of research, ...",Ayahuasca,Colorado
3,,DrugNerds,Vitamin D4 in Mushrooms
4,"Hi, sorry to bring this up, I am planning to t...",Ayahuasca,Is Pulse tours still open for business?


### Now we will prepare our data and create the features for modeling. Create 'X' column by CountVectorizer and target column from subreddit: 1 if it's 'DrugNerds' and 0 is it's 'Ayahuasca'.


In [334]:
df['X'] = df['selftext'] + df['title']

In [335]:
stop = list(ENGLISH_STOP_WORDS)
stop.extend(['https','amp'])

cv = CountVectorizer(min_df=2 ,stop_words=stop)


cv_arr = cv.fit_transform(df['X'])

df_vect = pd.DataFrame(cv_arr.toarray(), columns=cv.get_feature_names())
df_vect.head()

,00,000,00228,0025790,006,01,018,02,03,04,...,zebrafish,zero,zfl2014,zhuan,zinc,zombie,zone,α1receptors,δ9,μm
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [336]:
df_vect.shape

(1955, 7580)

In [337]:
df.subreddit = df.subreddit.map({'DrugNerds':1, 'Ayahuasca':0})
y = df['subreddit']

### Now let's try logistic regression to estimate result of count vectorizing.

In [338]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_vect,
                                                    y,
                                                    random_state = 42)

In [339]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [340]:
lr.score(X_train, y_train)

0.9924965893587995

In [341]:
lr.score(X_test, y_test)

0.9713701431492843

In [342]:
kf = KFold(n_splits=5, shuffle=True,random_state=42)
logreg = LogisticRegression(random_state=42)
cross_val_score(logreg,X_train,y_train,cv=kf).mean()

0.9317800840472709

In [343]:
cross_val_score(logreg,X_test,y_test,cv=kf).mean()

0.9365663791289712

### Let's try and create X from 'X' column by TfidVectorizer and use Logistic Regression again.

In [373]:
tfid = TfidfVectorizer(min_df=2, stop_words='english')

tfid_fit = tfid.fit_transform(df['X'])

df_tfid  = pd.DataFrame(tfid_fit.toarray(),
                   columns=tfid.get_feature_names())
df_tfid.head()

,00,000,00228,0025790,006,01,018,02,03,04,...,zebrafish,zero,zfl2014,zhuan,zinc,zombie,zone,α1receptors,δ9,μm
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [374]:
cross_val_score(logreg,df_tfid,y,cv=kf).mean()

0.941687979539642

### Now let's try Random Forests!

In [344]:
df['subreddit'].value_counts(normalize = True)

0    0.512532
1    0.487468
Name: subreddit, dtype: float64

#### 0.512532 is the baseline accuracy for our model.

Let's try and beat it.




In [377]:
rf = RandomForestClassifier(random_state=42)
cross_val_score(rf,df_vect,y,cv=kf).mean()

0.9140664961636829

In [378]:
cross_val_score(rf,df_tfid,y,cv=kf).mean()

0.9350383631713555

### KNeighbors Classifier

In [380]:
knn_params = {
    'n_neighbors': [5,15,21],
    'weights': ['distance'],
    'metric': ['manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(),
                    knn_params,
                    cv=3,
                    verbose = 1,
                   return_train_score = True)

grid.fit(df_vect, y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.6min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [5, 15, 21], 'weights': ['distance'], 'metric': ['manhattan']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [381]:
grid.best_score_

0.8721227621483376

In [382]:
knn_params = {
    'n_neighbors': [5,15,21],
    'weights': ['distance'],
    'metric': ['manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(),
                    knn_params,
                    cv=3,
                    verbose = 1,
                   return_train_score = True)

grid.fit(df_tfid, y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.9min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [5, 15, 21], 'weights': ['distance'], 'metric': ['manhattan']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [383]:
grid.best_score_

0.8040920716112532

### Finally let's try Voting Classifier...

In [346]:
knn_pipe = Pipeline([
    ('ss', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

vote = VotingClassifier([
    ('knn', knn_pipe),
    ('lr', LogisticRegression()),
    ('ada', AdaBoostClassifier()),
    ('gb', GradientBoostingClassifier())
])
vote.fit(df_vect, y)

VotingClassifier(estimators=[('knn', Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])), ('l...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [347]:
vote.score(df_vect, y)

0.9597544338335607

In [348]:
vote.score(X_test, y_test)

0.9652351738241309

In [384]:
knn_pipe = Pipeline([
    ('ss', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

vote = VotingClassifier([
    ('knn', knn_pipe),
    ('lr', LogisticRegression()),
    ('ada', AdaBoostClassifier()),
    ('gb', GradientBoostingClassifier())
])
vote.fit(df_tfid, y)

VotingClassifier(estimators=[('knn', Pipeline(memory=None,
     steps=[('ss', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])), ('l...      presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [385]:
vote.score(df_tfid, y)

0.9759590792838875

### Looks like Count Vectorizer and Logistic Regression model won by showing 0.9924 score on training data and 0.9713 score on testing. But also TfidVectorizer and VotingClassifier showed 0.9759 score, which is also very nice. The results of 2 best performing models totaly help us to resolve our problem.

# Executive Summary

We can see that two categories 'DrugNerd' and 'Ayahuasca' may be divided with very high accuracy. Among all the methods, logistic regression gave us the highest accuracy. That shows a lot about what people think and say about ayahuasca, definitely separating it from “drugs” category. We can see that people use more words like "experience", "ceremony", "retreat" and "help" when talking about Ayahuasca, that totally confirms that idea.